In [1]:
from write_to_tf import * 


1 Physical GPUs, 1 Logical GPUs


2023-09-27 14:26:47.017848: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21336 MB memory:  -> device: 0, name: Quadro RTX 6000, pci bus id: 0000:25:00.0, compute capability: 7.5


In [2]:
folder = "/uio/hume/student-u37/fslippe/data/nird_mount/MOD02/daytime_1km/"
start = "20191201"
end = "20210430"
bands = [6, 7, 20, 28, 29, 31]
start_converted = convert_to_day_of_year(start)
end_converted = convert_to_day_of_year(end)
x = [xi for xi in  extract_1km_data(folder, bands=bands, start_date=start_converted, end_date=end_converted) if xi.shape[0] > 64]
# Example usage:
print(x[1].dtype)
patch_size = 64

autoencoder = SimpleAutoencoder(len(bands), patch_size, patch_size)
patches = np.concatenate([autoencoder.extract_patches(n_d) for n_d in x], axis=0)

print(patches.dtype)


/uio/hume/student-u37/fslippe/data/nird_mount/MOD02/daytime_1km/MOD021KM.A2020031.0855.061.2020031192728.hdf
0 Latitude
1 Longitude
2 EV_1KM_RefSB
3 EV_1KM_RefSB_Uncert_Indexes
4 EV_1KM_Emissive
5 EV_1KM_Emissive_Uncert_Indexes
6 EV_250_Aggr1km_RefSB
7 EV_250_Aggr1km_RefSB_Uncert_Indexes
8 EV_250_Aggr1km_RefSB_Samples_Used
9 EV_500_Aggr1km_RefSB
10 EV_500_Aggr1km_RefSB_Uncert_Indexes
11 EV_500_Aggr1km_RefSB_Samples_Used
12 Height
13 SensorZenith
14 SensorAzimuth
15 Range
16 SolarZenith
17 SolarAzimuth
18 gflags
19 EV_Band26
20 EV_Band26_Uncert_Indexes
21 Band_250M
22 Band_500M
23 Band_1KM_RefSB
24 Band_1KM_Emissive
25 Noise in Thermal Detectors
26 Change in relative responses of thermal detectors
27 DC Restore Change for Thermal Bands
28 DC Restore Change for Reflective 250m Bands
29 DC Restore Change for Reflective 500m Bands
30 DC Restore Change for Reflective 1km Bands


  3%|▎         | 2/73 [03:28<2:03:22, 104.26s/it]


In [3]:
#print(patches.max(axis=(0,1,2)))

In [15]:
np.save("/scratch/fslippe/modis/MOD02/training_data/patches_202012-202104_1", patches[:40000])


In [ ]:

patches_per_file = 250000  # example value, you can adjust it

for i in range(0, len(patches), patches_per_file):
    print(i)
    chunk = patches[i: i+patches_per_file]
    write_tfrecord(f'/scratch/fslippe/modis/MOD02/training_data/tf_data/normalized_trainingpatches_band(1)_winter20_21_{i//patches_per_file}.tfrecord', chunk)
